In [ ]:
import json
from flask import Flask, request, jsonify
import requests

app = Flask(__name__)

# Define the URL of the external API
external_api_url = "http://localhost:11434/api/generate"

# Define headers for the external API request
headers = {'Content-Type': 'application/json'}

def generate_response(input_data):
    try:
        # Prepare prompt for the external API request
        prompt = f"Write a {input_data['format']} about {input_data['topic']} using emojis! \n"
        input_data["prompt"] = prompt

        # Prepare data for the external API request
        data = {
            "model": "llama3",
            "stream": False,
            "prompt": json.dumps(input_data),
        }

        # Send a POST request to the external API
        response = requests.post(external_api_url, headers=headers, json=data)

        # Check if the request was successful
        if response.status_code == 200:
            # Extract the response from the external API
            response_json = response.json()
            actual_response = response_json.get("response", "No response")
            return actual_response
        else:
            # Print an error message if the request fails
            print("Error:", response.status_code, response.text)
            return f"Error: {response.status_code}"

    except Exception as e:
        print("Error sending request to external API:", e)
        return f"Error: {str(e)}"


@app.route('/api/generate_response', methods=['POST'])
def generate_response_api():
    try:
        # Get input data from the request
        input_data = request.json

        # Check if 'format' and 'topic' are present in the input data
        if "format" not in input_data or "topic" not in input_data:
            return jsonify({"error": "Missing 'format' or 'topic' in input data"}), 400

        # Generate response using external API
        output = generate_response(input_data)

        # Return the response
        return jsonify({"response": output})

    except Exception as e:
        return jsonify({"error": str(e)}), 500


@app.route('/')
def index():
    return "Flask API is running!"

if __name__ == "__main__":
    app.run(debug=True)
